# Lane Detection and Obstacle Avoidance 

### Load the pre-trained models

In [1]:
import torchvision
import torch

steering_model = torchvision.models.resnet18(pretrained=False)
steering_model.fc = torch.nn.Linear(512, 2)

collision_model = torchvision.models.alexnet(pretrained=False)
collision_model.classifier[6] = torch.nn.Linear(collision_model.classifier[6].in_features, 2)

In [2]:
steering_model.load_state_dict(torch.load('best_steering_model_xy_v1.pth'))
collision_model.load_state_dict(torch.load('best_model.pth'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

### Use the Jetbot GPU

In [3]:
device = torch.device('cuda')
steering_model = steering_model.to(device)
collision_model = collision_model.to(device)
steering_model = steering_model.eval().half()

### Define the preprocessing functions

In [4]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [5]:
mean2 = 255.0 * np.array([0.485, 0.456, 0.406])
stdev2 = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean2, stdev2)

def preprocess_blocked(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

### Create a bot instance and define the speed and steering sliders

In [6]:
from jetbot import Robot
robot = Robot()

In [7]:
import ipywidgets
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')
blocked_slider = ipywidgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider,blocked_slider)

FloatSlider(value=0.0, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.2, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.0, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

FloatSlider(value=0.0, description='blocked', max=1.0, orientation='vertical')

In [8]:
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='y')
left_motor = ipywidgets.FloatText(description='left motor')
right_motor = ipywidgets.FloatText(description='right motor')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

### Define the processing function that will predict the obsctacle state and steer the bot

In [27]:
angle = 0.0
angle_last = 0.0
left_motor_value = 0.0
right_motor_value = 0.0
from jetbot import Camera, bgr8_to_jpeg
import time

camera = Camera(fps=6)

def execute(change):
    global angle, angle_last, blocked_slider, robot
    image = change['new']
    xy = steering_model(preprocess(image)).detach().float().cpu().numpy().flatten()
    pred = collision_model(preprocess_blocked(image))
    prob_blocked = 0
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    pred = F.softmax(pred, dim=1)
    
    prob_blocked = float(pred.flatten()[0])
    
    blocked_slider.value = prob_blocked
    
            
    x = xy[0]
    y = (0.5 - xy[1]) / 2.0

    x_slider.value = x
    y_slider.value = y

    speed_slider.value = speed_gain_slider.value

    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle

    steering_slider.value = pid + steering_bias_slider.value
    left_motor_value = max(min(speed_slider.value + steering_slider.value, 1.0), 0.0)
    right_motor_value = max(min(speed_slider.value - steering_slider.value, 1.0), 0.0)

    if prob_blocked > 0.7:
        robot.stop()
    else:
        robot.left_motor.value = left_motor_value
        robot.right_motor.value = right_motor_value
    
execute({'new': camera.value})

### Display the model predection

In [28]:
from IPython.display import display

import traitlets
display(blocked_slider)

FloatSlider(value=0.179335355758667, description='blocked', max=1.0, orientation='vertical')

### Start the action

In [29]:
camera.observe(execute, names='value')

### Stop the bot and remove the camera reference 

In [30]:
import time

camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

In [ ]:
camera_link.unlink()

In [31]:
camera.stop()